In [ ]:
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer
from soynlp.hangle import jamo_levenshtein\


In [ ]:
from kor2vec import Kor2Vec

In [ ]:
import torch.nn as nn

In [ ]:
import re
import codecs
import pandas as pd
import pickle

In [ ]:
EMBEDDING_DIM = 128
HIDDEN_SIZE = 10
LABEL_SIZE = 7

In [7]:
class TrainModel():
    def __init__(self, fileName, 
                 vocabFileName = "./korean_embedding/vocab.txt", tokenizerFileName = "./korean_embedding/tokenizer.pkl", 
                 kor2vecFileName = "./korean_embedding/embedding.model"):
        
        self.fileName = fileName
        self.vocabFileName = vocabFileName
        self.tokenizerFileName = tokenizerFileName
        self.kor2vecFileName = kor2vecFileName       
       
        self.readNLP()
        self.readDataset()
        
        self.loss_function = nn.NLLLoss()
        self.optimizer = optim.SGD(model.parameters(), lr=0.1)
        
        self.model = SentenceClassifier()
        self.trainModel()
    # 매개변수로 받은 sentence에서 문장부호를 제외한 한글만 남김
    def onlyKorean(self, sentence):    
        korean = re.compile('[^ ㄱ-ㅣ가-힣]+') 
        result = korean.sub('', sentence)
        return result

    def readDataset(self):
        self.sentence = pd.read_excel(self.fileName + '.xlsx')['question']
        print(' read question data from ', self.fileName)        
        for i in range(0,len(sentence)):
            self.sentence[i] = self.onlyKorean(sentence[i])

        print('delete punctuation marks from data')
        
    def readNLP(self):
        # tokenizer 사용하는 이유 = 띄어쓰기 문제 해결을 위하여
        with open('./korean_embedding/tokenizer.pkl','rb') as f:
            self.tokenizer = pickle.load(f)
        # model
        self.kor2vec = Kor2Vec.load("./korean_embedding/embedding.model")
        # vocab
        self.vocab = []
        f = open("./korean_embedding/vocab.txt", 'r')
        while True:
            word = f.readline()
            if not word: 
                break
            else :
                self.vocab.append(word[:-1])
        f.close()
        
    def trainModel(self):
        # training
        for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
            for sentence, tags in training_data:
                # Step 1. Remember that Pytorch accumulates gradients.
                # We need to clear them out before each instance
                model.zero_grad()

                # Also, we need to clear out the hidden state of the LSTM,
                # detaching it from its history on the last instance.
                model.hidden = model.init_hidden()

                # Step 2. Get our inputs ready for the network, that is, turn them into
                # Tensors of word indices.
                sentence_in = prepare_sequence(sentence, word_to_ix)
                targets = prepare_sequence(tags, tag_to_ix)

                # Step 3. Run our forward pass.
                tag_scores = model(sentence_in)

                # Step 4. Compute the loss, gradients, and update the parameters by
                #  calling optimizer.step()
                loss = loss_function(tag_scores, targets)
                loss.backward()
                optimizer.step()

SyntaxError: unexpected EOF while parsing (<ipython-input-7-8b6fde7d4865>, line 2)

In [ ]:
# embedding 모델은 따로
class SentenceClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, label_size):
        super(SentenceClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
               torch.zeros(1, 1, self.hidden_dim))
    
    # x = embedding.vectorizeSentence(list of sentence)
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y = self.hidden2label(lstm_out)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)

In [ ]:
kor2vec = Kor2Vec.load("../model/path")
# or kor2vec = SejongVector()

lstm = nn.LSTM(128, 64, batch_first=True)
dense = nn.Linear(64, 1)

# Make tensor input
sentences = ["이 영화는 정말 대박이에요", "우와 진짜 재미있었어요"]

x = kor2vec.to_seqs(sentences, seq_len=10)
# >>> tensor(batch_size, seq_len, char_seq_len)

x = kor2vec(x) # tensor(batch_size, seq_len, 128)
_, (x, xc) = lstm(x) # tensor(batch_size, 64)
x = dense(x) # tensor(batch_size, 1)

In [ ]:
# test = vocab.kor2vec.embedding("김동호 교수님 수업 어때?")

# input = vocab.kor2vec.to_seqs(["김동호 교수님 수업 어때?", "컴퓨터보안"], seq_len=6)
# vocab.kor2vec.forward(input)

In [ ]:
# dataset(train/test) 구성

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

model = SentenceClassifier()